# Working on the Datasets

We will use the pandas library to read the datasets. We will use the `read_csv` method to read the csv files in python and assign it to a dataframe object.

### Required Libraries

- pandas
- matplotlib
- numpy
- scikit-learn


To install the libraries, you can use `pip` or `conda` as follows:

```bash
pip install pandas matplotlib scikit-learn numpy
```

```bash
conda install pandas matplotlib scikit-learn numpy
```


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

### About the dataset

This dataset can be found on the following [link](https://www.kaggle.com/datasets/blastchar/telco-customer-churn).

The dataset contains 7043 rows and 21 columns.

The dataset contains information about a telecom company that has a problem with the churn rate of its customers. The company wants to know which customers are likely to churn next month. The dataset contains information about:

- Customers who left within the last month – the column is called Churn
- Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
- Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
- Demographic info about the customers

In [ ]:
dataset = pd.read_csv('Telco-Customer-Churn.csv')
# dataset = pd.read_csv('short.csv')

# replace the cells with single and extra spaces with a single space
dataset = dataset.replace(r'\s+', ' ', regex=True)

# replace the single space with a NaN value
dataset = dataset.replace(r'^\s*$', np.nan, regex=True)

# drop the id column from the dataset
dataset = dataset.drop('customerID', axis=1)

# How many rows and columns are in this dataset?
print(dataset.shape)

# Store the column of data types in a list called data_types
data_types = dataset.dtypes

# Print the data type of each column
print(data_types)

# What is the first entry in the dataset?
print(dataset.head(2))

# What is the last entry in the dataset?
print(dataset.tail(2))

# What is the name of each column?
print(dataset.columns)

### Converting the yes and no values to 1 and 0

We first will scan for those columns that have yes and no values and convert them to 1 and 0 respectively.

**Note:** This cell is dependent on the dataset and the column names. If you are using a different dataset, you will need to change the column names.

In [ ]:
dataset = pd.DataFrame(dataset)

# Name of the column that needs to be encoded in 0s and 1s
column_name = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']

# Name of the column that is a categorical variable
categorical_col_names = ['Gender', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']


# Create a label encoder object
le = LabelEncoder()

# Apply the label encoder object on the data
dataset[column_name] = dataset[column_name].apply(le.fit_transform)

# Print the first 5 rows of the dataset
# print(dataset.head(5))

print(dataset.dtypes)


### Dividing the dataset into dependent and independent variables

We will divide the dataset into dependent and independent variables. 

The dependent variable is the variable that we want to predict.The independent variables are the variables that we will use to predict the dependent variable. In this case, 

- It is the `Churn` column. 
- All the other columns except the `Churn` column and `Customer ID` column.

In [ ]:
# divide the dataset into x and y
dataset_columns_length = len(dataset.columns)
print(dataset_columns_length)

x = dataset.iloc[:, 0:(dataset_columns_length-1)].values
y = dataset.iloc[:, (dataset_columns_length-1)].values

# What is the shape of x and y?
print(x.shape)
print(y.shape)


# What is the value of the 1st row of x?
print(x[0])

# What is the value of the 1st row of y?
print(y[0])

### Checking for missing values

We will check for missing values in the dataset. If there are any missing values, we will replace them with the mean of the column.

In [ ]:
dataset.isnull().sum()   # check for missing values

# Store the column number of the columns with missing values in a list called missing_cols
missing_cols = [i for i in range(dataset_columns_length) if dataset.iloc[:, i].isnull().any()]
print(missing_cols)

# Print columns index and names with missing values and the number of missing values
for i in missing_cols:
    print(i, dataset.columns[i], dataset.iloc[:, i].isnull().sum())


In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# impute missing values in the columns with missing values
# Here the hard coded values are the column numbers of the columns with missing values 
x[:, 18] = imputer.fit_transform(x[:, 18].reshape(-1, 1)).ravel()

# print(x[488])

# print the max and min values of the column with missing values
print(max(x[:, 18]))
print(min(x[:, 18]))


### Re check for the datatypes of the each column in x, y

We need to recheck the datatypes of the columns in x and y. If there are any columns that are not numeric, we will convert them to numeric.

In [ ]:
# length of the independent variable dataset
x_length = len(x[0])

# find the list of missing columns in the independent variable dataset
missing_cols_x = [i for i in range(x_length) if pd.isnull(x[:, i]).any()]

print(missing_cols_x)

# Print columns index and names with missing values and the number of missing values
for i in missing_cols_x:
    print(i, dataset.columns[i], pd.isnull(x[:, i]).sum())
    
    
# data types of the independent variable dataset
# for each column of independent variable dataset, print the data type
for i in range(x_length):
    print(x[:, i])
    
y_data_types = y.dtype

# Print the data type of each column
print(y_data_types)

    

## Check for the categorical columns

- We need to check for the categorical columns in the dataset.
- For this we will use the `one hot encoding` technique. 

In [ ]:
# find the list of the columns with the categorical data in the independent variable dataset
# here the value is hard coded
categorical_col_index = [0, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16]

# one hot encoder object 
# the column names should be passed as a parameter
one_hot_encoder = OneHotEncoder(dtype=np.int64, handle_unknown='ignore')

# apply the one hot encoder object on the independent variable dataset
encoded_x = one_hot_encoder.fit_transform(x[:, categorical_col_index]).toarray()

print(one_hot_encoder.get_feature_names_out(input_features=categorical_col_names))

# print the first row of the independent variable dataset
# print(x[0])
print(encoded_x[0])

print(encoded_x.shape)

- Now we need to drop the original columns and add the new columns to the dataset.

In [ ]:
# drop the categorical columns from the independent variable dataset
x = np.delete(x, categorical_col_index, axis=1)

print(x.shape)

# concatenate the encoded columns with the independent variable dataset
x = np.concatenate((x, encoded_x), axis=1)

print(x.shape)